In [16]:
!pip install -q neo4j 
!pip install -q dotenv

In [17]:
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Neo4j connection configuration from environment variables
uri = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USER")
password = os.getenv("NEO4J_PASSWORD")

# Create driver and connect to database
driver = GraphDatabase.driver(uri, auth=(username, password))

# Test connection
def test_connection():
    with driver.session() as session:
        result = session.run("RETURN 'Connection successful!' as message")
        return result.single()["message"]

try:
    message = test_connection()
    print(message)
except Exception as e:
    print(f"Connection failed: {e}")

Connection successful!


#### Thinking behind the data.
- text: the text body.
- ConversationID = what coversation.
- MessageID num ID in the conversation.
- What part of the process is the user.
- user: the users name
- userID: Unique user id
- created at time stamp
- role: agent or user

In [18]:
def insert_hello_message():
    with driver.session() as session:
        query = """
        CREATE (m:Message {
            text: 'hello',
            conversation_id: 1,
            message_id: 1,
            plan_id: 1,
            user: 'morris',
            created_at: datetime(),
            role: 'user'
        })
        RETURN m
        """
        result = session.run(query)
        return result.single()

# Insert the message
try:
    message_node = insert_hello_message()
    print("Message inserted successfully!")
    print(f"Created node: {message_node['m']}")
except Exception as e:
    print(f"Failed to insert message: {e}")

Message inserted successfully!
Created node: <Node element_id='4:ca7b4fdd-56c0-4663-a686-ea51332c9fec:349' labels=frozenset({'Message'}) properties={'role': 'user', 'conversation_id': 1, 'created_at': neo4j.time.DateTime(2025, 5, 26, 20, 11, 16, 288000000, tzinfo=<UTC>), 'message_id': 1, 'text': 'hello', 'user': 'morris', 'plan_id': 1}>


In [19]:
def get_all_nodes():
    with driver.session() as session:
        query = "MATCH (n) RETURN n"
        result = session.run(query)
        return [record["n"] for record in result]

# Get all nodes
try:
    all_nodes = get_all_nodes()
    print(f"Found {len(all_nodes)} nodes:")
    for node in all_nodes:
        print(node)
except Exception as e:
    print(f"Failed to retrieve nodes: {e}")

Found 1 nodes:
<Node element_id='4:ca7b4fdd-56c0-4663-a686-ea51332c9fec:349' labels=frozenset({'Message'}) properties={'role': 'user', 'conversation_id': 1, 'created_at': neo4j.time.DateTime(2025, 5, 26, 20, 11, 16, 288000000, tzinfo=<UTC>), 'message_id': 1, 'text': 'hello', 'user': 'morris', 'plan_id': 1}>


In [23]:
def insert_user_message():
    with driver.session() as session:
        query = """
        CREATE (m:Message {
            text: 'hello how can i help you? Lorem Ipsum is a standard placeholder text used in publishing and web design to fill spaces where content will eventually go. It originates from a scrambled section of Ciceros De Finibus Bonorum et Malorum written in 45 BC. Designers use it to focus on layout and visual elements without the distraction of meaningful content',
            conversation_id: 1,
            message_id: 2,
            plan_id: 1,
            user: 'Frida',
            created_at: datetime(),
            role: 'assistant'
        })
        RETURN m
        """
        result = session.run(query)
        return result.single()

# Insert the message with T-agent user
try:
    user_message_node = insert_user_message()
    print("Message with T-agent user inserted successfully!")
    print(f"Created node: {user_message_node['m']}")
except Exception as e:
    print(f"Failed to insert message: {e}")

Message with T-agent user inserted successfully!
Created node: <Node element_id='4:ca7b4fdd-56c0-4663-a686-ea51332c9fec:350' labels=frozenset({'Message'}) properties={'role': 'assistant', 'conversation_id': 1, 'created_at': neo4j.time.DateTime(2025, 5, 26, 20, 12, 51, 184000000, tzinfo=<UTC>), 'message_id': 2, 'text': 'hello how can i help you? Lorem Ipsum is a standard placeholder text used in publishing and web design to fill spaces where content will eventually go. It originates from a scrambled section of Ciceros De Finibus Bonorum et Malorum written in 45 BC. Designers use it to focus on layout and visual elements without the distraction of meaningful content', 'user': 'Frida', 'plan_id': 1}>


In [24]:
def create_conversation_relationships():
    with driver.session() as session:
        query = """
        MATCH (m1:Message), (m2:Message)
        WHERE m1.conversation_id = m2.conversation_id AND m1 <> m2
        CREATE (m1)-[:SAME_CONVERSATION]->(m2)
        RETURN count(*) as relationships_created
        """
        result = session.run(query)
        return result.single()["relationships_created"]

# Create relationships between messages in the same conversation
try:
    count = create_conversation_relationships()
    print(f"Created {count} conversation relationships!")
except Exception as e:
    print(f"Failed to create relationships: {e}")

Created 2 conversation relationships!


In [29]:
def create_vector_index():
    with driver.session() as session:
        query = """
        CREATE VECTOR INDEX message_text_vector IF NOT EXISTS
        FOR (m:Message) ON (m.text_vector)
        OPTIONS {
          indexConfig: {
            `vector.dimensions`: 384,
            `vector.similarity_function`: 'cosine'
          }
        }
        """
        result = session.run(query)
        return result.consume()

# Create vector index for message text
try:
    create_vector_index()
    print("Vector index 'message_text_vector' created successfully!")
except Exception as e:
    print(f"Failed to create vector index: {e}")

Vector index 'message_text_vector' created successfully!


In [31]:
def get_vector_search_index():
    with driver.session() as session:
        query = "SHOW INDEXES YIELD name, type WHERE type = 'VECTOR'"
        result = session.run(query)
        return [record for record in result]

# Check for existing vector indexes
try:
    vector_indexes = get_vector_search_index()
    if vector_indexes:
        print("Existing vector indexes:")
        for index in vector_indexes:
            print(f"Index name: {index['name']}, Type: {index['type']}")
    else:
        print("No vector indexes found. You may need to create one first.")
except Exception as e:
    print(f"Failed to retrieve vector indexes: {e}")

Existing vector indexes:
Index name: message_text_vector, Type: VECTOR


In [32]:
def vector_search_similar_messages(query_text, top_k=5):
    with driver.session() as session:
        # First, we need to get embeddings for the query text
        # For this example, I'll show how to do vector search assuming you have embeddings
        query = """
        MATCH (m:Message)
        WHERE m.text_vector IS NOT NULL
        CALL db.index.vector.queryNodes('message_text_vector', $top_k, $query_vector)
        YIELD node, score
        RETURN node.text as text, node.user as user, node.role as role, score
        ORDER BY score DESC
        """
        # Note: You'll need to generate embeddings for query_text first
        # This is a placeholder - you'll need actual vector embeddings
        result = session.run(query, top_k=top_k, query_vector=[0.1] * 384)
        return [record for record in result]

# Alternative: Simple text similarity search without vectors
def search_similar_text(search_term):
    with driver.session() as session:
        query = """
        MATCH (m:Message)
        WHERE toLower(m.text) CONTAINS toLower($search_term)
        RETURN m.text as text, m.user as user, m.role as role, m.message_id as message_id
        ORDER BY m.message_id
        """
        result = session.run(query, search_term=search_term)
        return [record for record in result]

# Search for messages containing "hello"
try:
    similar_messages = search_similar_text("hello")
    print(f"Found {len(similar_messages)} messages containing 'hello':")
    for msg in similar_messages:
        print(f"User: {msg['user']}, Role: {msg['role']}")
        print(f"Text: {msg['text'][:100]}...")
        print("---")
except Exception as e:
    print(f"Search failed: {e}")

Found 2 messages containing 'hello':
User: morris, Role: user
Text: hello...
---
User: Frida, Role: assistant
Text: hello how can i help you? Lorem Ipsum is a standard placeholder text used in publishing and web desi...
---
